# Restaurants reviews

In [1]:
import requests
import pandas as pd

# Set your Google Maps API key
api_key = ''

# Set the location (Amsterdam)
location = 'Amsterdam'

# Set the type of place (restaurant)
place_type = 'restaurant'

# Set up the API endpoint
endpoint = 'https://maps.googleapis.com/maps/api/place/textsearch/json'
params = {
    'query': f'{place_type} in {location}',
    'key': api_key
}

# Make the API request
response = requests.get(endpoint, params=params)
results = response.json().get('results', [])

# Create lists to store details
places_data = []
reviews_data = []

# Function to get details for a place
def get_place_details(place_id):
    details_endpoint = 'https://maps.googleapis.com/maps/api/place/details/json'
    details_params = {
        'place_id': place_id,
        'key': api_key,
    }

    details_response = requests.get(details_endpoint, params=details_params)
    details_result = details_response.json().get('result', {})

    # Extract required details
    formatted_address = details_result.get('formatted_address', '')
    phone_number = details_result.get('formatted_phone_number', '')
    website = details_result.get('website', '')
    about = details_result.get('formatted_address', '')
    allows_dogs = 'Yes' if 'dogs' in details_result.get('types', []) else 'No'

    # Extract reviews details
    for review in details_result.get('reviews', []):
        author_name = review.get('author_name', '')
        rating = review.get('rating', '')
        text = review.get('text', '')
        date = review.get('relative_time_description', '')

        reviews_data.append({
            'place_name': details_result.get('name', ''),
            'formatted_address': formatted_address,
            'phone_number': phone_number,
            'website': website,
            'about': about,
            'allowsDogs': allows_dogs,
            'author_name': author_name,
            'rating': rating,
            'text': text,
            'date': date
        })

# Iterate through the results and get details for each place
for result in results[:100]:  # Limit to the first 100 results
    place_id = result.get('place_id')
    get_place_details(place_id)

# Create DataFrames
reviews_df = pd.DataFrame(reviews_data)

# Save DataFrames to CSV
reviews_df.to_csv('restaurants_reviews_details.csv', index=False)

# Display the DataFrames
#print(reviews_df)


In [2]:
reviews_df.describe()

,rating
count,100.000000
mean,4.700000
std,0.689019
min,1.000000
25%,5.000000
50%,5.000000
75%,5.000000
max,5.000000


In [3]:
import pandas as pd

# Read the merged CSV file into a DataFrame
merged_df = pd.read_csv('restaurants_reviews_details.csv')

# Function to check if a review mentions being pet-friendly
def is_friendly(text):
    friend_keywords = ['friend']
    return any(keyword in text.lower() for keyword in friend_keywords)

# Add a new column 'pet_friendly' with 'Yes' or 'No' based on the analysis
merged_df['friendly'] = merged_df['text'].apply(lambda x: 'Yes' if is_friendly(x) else 'No')

# Display the updated DataFrame
print(merged_df)

# Save the updated DataFrame to a new CSV file
merged_df.to_csv('restaurants_reviews_details_friendly.csv', index=False)


                             place_name  \
0                    Restaurant Olijfje   
1                    Restaurant Olijfje   
2                    Restaurant Olijfje   
3                    Restaurant Olijfje   
4                    Restaurant Olijfje   
..                                  ...   
95  Saint Morris Argentinian Restaurant   
96  Saint Morris Argentinian Restaurant   
97  Saint Morris Argentinian Restaurant   
98  Saint Morris Argentinian Restaurant   
99  Saint Morris Argentinian Restaurant   

                                    formatted_address  phone_number  \
0   Valkenburgerstraat 223D, 1011 MJ Amsterdam, Ne...  020 330 4444   
1   Valkenburgerstraat 223D, 1011 MJ Amsterdam, Ne...  020 330 4444   
2   Valkenburgerstraat 223D, 1011 MJ Amsterdam, Ne...  020 330 4444   
3   Valkenburgerstraat 223D, 1011 MJ Amsterdam, Ne...  020 330 4444   
4   Valkenburgerstraat 223D, 1011 MJ Amsterdam, Ne...  020 330 4444   
..                                                ...    

In [18]:
# Filter rows where pet_friendly is 'Yes'
pet_friendly_df = merged_df[merged_df['pet_friendly'] == 'Yes']

In [19]:
pet_friendly_df

,place_name,formatted_address,phone_number,website,about,allowsDogs,author_name,rating,text,date,pet_friendly
